In [1]:
!pip install yfinance
!pip install yahooquery
from yahooquery import Screener
import yfinance as yf   
import pandas as pd
import numpy as np
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
import random 
from tensorflow.keras.models import load_model
import os
import shutil
from datetime import datetime
import glob

     |████████████████████████████████| 6.4 MB 12.7 MB/s 
     |████████████████████████████████| 63 kB 819 kB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.27.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
     |████████████████████████████████| 46 kB 2.4 MB/s 


In [351]:
clmns = [
    "Open 12-11","Open 11-10","Open 10-9","Open 9-8","Open 8-7","Open 7-6","Open 6-5","Open 5-4","Open 4-3","Open 3-2","Open 2-1",
    "Close 12-11","Close 11-10","Close 10-9","Close 9-8","Close 8-7","Close 7-6","Close 6-5","Close 5-4","Close 4-3","Close 3-2","Close 2-1",
    "High 12-11","High 11-10","High 10-9","High 9-8","High 8-7","High 7-6","High 6-5","High 5-4","High 4-3","High 3-2","High 2-1",
    "Low 12-11","Low 11-10","Low 10-9","Low 9-8","Low 8-7","Low 7-6","Low 6-5","Low 5-4","Low 4-3","Low 3-2","Low 2-1",
    "Volume 12-11","Volume 11-10","Volume 10-9","Volume 9-8","Volume 8-7","Volume 7-6","Volume 6-5","Volume 5-4","Volume 4-3","Volume 3-2","Volume 2-1",
    "suggestion"]
def work_with_dir():
  if os.path.exists("/content/data/"):
    shutil.rmtree("/content/data/", ignore_errors=True)
    print("Data Folder Removed")
    os.mkdir("/content/data/")

  if not os.path.exists("/content/data/"):
    os.mkdir("/content/data/")
  
  if not os.path.exists("/content/extracted/"):
    os.mkdir("/content/extracted/")
def read_txt_list():
  with open("yahoo_stocklist.txt","r")as f:
    lines = f.readlines()
    nlines = []
    for line in lines:
       nlines.append(line.strip())
    return nlines
def read_syms_from_txt():
  with open("syms.txt", "r") as f:
    lines = f.readlines()
    f.close()
  lst = []
  for line in lines:
    line = line.strip()
    lst.append(line)
  symbols = lst
  return symbols
def get_crypto_syms():
   # 'all_cryptocurrencies_au','all_cryptocurrencies_ca','all_cryptocurrencies_eu','all_cryptocurrencies_gb','all_cryptocurrencies_in',
   screens = [
       'all_cryptocurrencies_us', 'all_cryptocurrencies_au', 'all_cryptocurrencies_ca', 'all_cryptocurrencies_eu', 'all_cryptocurrencies_gb', 'all_cryptocurrencies_in', ]
   s = Screener()
   symbols = []
   for i in screens:
      data = s.get_screeners(i, count=250)
      dicts = data[i]['quotes']
      syms = [d['symbol'] for d in dicts]
      for sym in syms:
        symbols.append(sym)
   # print(len(symbols))
   # pieces = 15
   # new_arrays = np.array_split(symbols, pieces)
   return symbols
def spliting(data):
  X = data.drop(["yes","no"], axis=1)
  y = data[["yes","no"]]
  xTrain, xTest, yTrain, yTest = train_test_split(X, y, test_size=0.2)
  print(xTrain.shape, end=" ")
  print(yTrain.shape)
  print(xTest.shape, end=" ")
  print(yTest.shape)
  return xTrain, xTest, yTrain, yTest
def scaler(row):
    scaler = MinMaxScaler(feature_range=(-1, 1))
    row = scaler.fit_transform(row)
    return row
def process(data):
    data = data.dropna()
    row = []
    if len(data.columns) == 7:
      data = data.iloc[: , 1:]        
    data = np.array(data)
    llst = [0, 1, 2, 3, 5]
    for i in range(12, data.shape[0]):
        grow = []
        srow = []

        for j in llst:
           srow.append([
               data[i-1][j] - data[i-2][j],
               data[i-2][j] - data[i-3][j],
               data[i-3][j] - data[i-4][j],
               data[i-4][j] - data[i-5][j],
               data[i-5][j] - data[i-6][j],
               data[i-6][j] - data[i-7][j],
               data[i-7][j] - data[i-8][j],
               data[i-8][j] - data[i-9][j],
               data[i-9][j] - data[i-10][j],
               data[i-10][j] - data[i-11][j],
               data[i-11][j] - data[i-12][j]
           ])

        for lst in srow:
            mm = np.array(lst)
            mm = np.reshape(mm, (-1, 1))
            grow.append(scaler(mm))

        sugg = "no"
        if data[i][3] > data[i][0]:
            sugg = "yes"

        arr = np.array(grow).flatten()
        arr = np.append(arr, sugg)
        row.append(arr)


    grow = []
    srow = []
    llst = []
    data = []
    arr = []
    mm = []

    return np.array(row)

In [3]:
def download_data(symbols,periodd,intervall):
  
  indexx = 100
  work_with_dir()
  for symbol in symbols:
    if ((symbols.index(symbol)+1) % 100 == 0):
      print(f" -- {indexx}",end="")
      indexx = indexx + 100

    try:
        data = yf.download(symbol, period=periodd,interval=intervall, progress=False,show_errors=False)
        if data.empty:
           pass
        else:
           if data.shape[0] > 12:
             data.to_csv(f"/content/data/{symbol}.csv")
             
    except:
       print("Error!")
  print(" ")
def each_file_proc(files,now,index):
     data = []
     unattached_dfs = []
     files = list(files)
     for file in files:
        print(f"{files.index(file)+1+index}",end=" ")
        if (files.index(file)+index+1) % 40 == 0:
          print(" ")
        address = f"/content/data/{file}"
        unattached_dfs.append(process(pd.read_csv(address)))
     data = np.array(unattached_dfs[0])
     for i in unattached_dfs[1:]:
           data = np.append(data, np.array(i), axis=0)
        
     unattached_dfs = []
  
     data = pd.DataFrame(data, columns=clmns)
     sugg = data["suggestion"]
     data.drop("suggestion",axis=1,inplace=True)
     sugg = pd.get_dummies(sugg)
     data = pd.concat([data,sugg],axis=1)
     data = data.astype(float)
     right_now = datetime.now().strftime("%H%M%S%f")
     data.to_csv(f"/content/extracted/{now}/{right_now}.csv")  
def extract_data(pieces):
  pd.options.mode.chained_assignment = None
  print(f"Files In Data : {len(os.listdir('/content/data/'))}")
  files = os.listdir("/content/data/")
  new_files = np.array_split(files, pieces)
  print("Processing File:")
  now = datetime.now().strftime("%H%M%S")
  os.mkdir(f"/content/extracted/{now}/")
  
  index = 0 
  for files in new_files:
     
     each_file_proc(files,now,index)
     index = index + len(files)
  print(" ")
  return now
def delete_all_csv(now):
   path = f'/content/extracted/{now}/*.csv'
   files = glob.glob(path)
   for file in files:
       os.remove(file)
def make_df(now):
   path = f'/content/extracted/{now}/*.parquet'
   files = glob.glob(path)
   #data = pd.DataFrame()
   data = pd.DataFrame()
   for adr in files:
     data =pd.concat([data,pd.read_parquet(adr)])
   if "Unnamed: 0" in data:
     data.drop("Unnamed: 0",axis=1,inplace=True)
   print(data.shape)
   xTrain,xTest,yTrain,yTest = spliting(data)
   data.to_parquet(f'/content/extracted/{now}/data.parquet')
   delete_all_csv(now)
   data = []
   return xTrain,xTest,yTrain,yTest
def to_par(now,howmanyfiles): 
    files = os.listdir(f"/content/extracted/{now}/")
    addresses = []
    for file in files:
      addresses.append(f"/content/extracted/{now}/{file}")
    new_adr = np.array_split(addresses,howmanyfiles)
    for adrs in new_adr:
      datas = []
      for adr in adrs:
        datas.append(pd.read_csv(adr))
      rnow = datetime.now().strftime("%H%M%S%f")
      datas = pd.concat(datas)
      datas.to_parquet(f"/content/extracted/{now}/part_{rnow}.parquet")      

In [352]:
#symbols = get_crypto_syms()
#symbols = read_txt_list()
#symbols = read_syms_from_txt()
symbols = ["btc-usd","eth-usd","trx-usd"]

print(f"Symbols : {len(symbols)}")
download_data(symbols,"1d","5m")
folder_name = extract_data(1)
to_par(folder_name,1)
xTrain,xTest,yTrain,yTest = make_df(folder_name)

Symbols : 3
Data Folder Removed
 
Files In Data : 3
Processing File:
1 2 3  
(495, 57)
(396, 55) (396, 2)
(99, 55) (99, 2)


In [353]:
model = Sequential()



model.add(Dense(128, activation='relu', input_shape=(xTrain.shape[1],)))
model.add(Dense(128, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(128, activation='relu'))


opt = tf.keras.optimizers.Adamax()

model.add(Dense(2, activation='sigmoid'))
model.compile(optimizer=opt, loss='categorical_crossentropy',metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_5 (Dense)             (None, 128)               7168      
                                                                 
 dense_6 (Dense)             (None, 128)               16512     
                                                                 
 dense_7 (Dense)             (None, 128)               16512     
                                                                 
 dense_8 (Dense)             (None, 128)               16512     
                                                                 
 dense_9 (Dense)             (None, 2)                 258       
                                                                 
Total params: 56,962
Trainable params: 56,962
Non-trainable params: 0
_________________________________________________________________


In [354]:
model.fit(xTrain,yTrain,epochs=100,batch_size=10,validation_data=(xTest,yTest),shuffle=False)

Epoch 1/100
40/40 [==============================] - 1s 7ms/step - loss: 0.7032 - accuracy: 0.5253 - val_loss: 0.6960 - val_accuracy: 0.5354
Epoch 2/100
40/40 [==============================] - 0s 3ms/step - loss: 0.6455 - accuracy: 0.6465 - val_loss: 0.6935 - val_accuracy: 0.5657
Epoch 3/100
40/40 [==============================] - 0s 3ms/step - loss: 0.6100 - accuracy: 0.7020 - val_loss: 0.6868 - val_accuracy: 0.5556
Epoch 4/100
40/40 [==============================] - 0s 4ms/step - loss: 0.5685 - accuracy: 0.7424 - val_loss: 0.6812 - val_accuracy: 0.5758
Epoch 5/100
40/40 [==============================] - 0s 3ms/step - loss: 0.5188 - accuracy: 0.7727 - val_loss: 0.6806 - val_accuracy: 0.6162
Epoch 6/100
40/40 [==============================] - 0s 3ms/step - loss: 0.4608 - accuracy: 0.8106 - val_loss: 0.6873 - val_accuracy: 0.6263
Epoch 7/100
40/40 [==============================] - 0s 3ms/step - loss: 0.3958 - accuracy: 0.8510 - val_loss: 0.7074 - val_accuracy: 0.6263
Epoch 8/100
4

In [369]:
data = yf.download("trx-usd",period="1d",interval="5m")
data

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Datetime,,,,,,
2022-04-26 00:00:00+00:00,0.065323,0.065323,0.065303,0.065318,0.065318,0
2022-04-26 00:05:00+00:00,0.065294,0.065294,0.065235,0.065235,0.065235,362752
2022-04-26 00:10:00+00:00,0.065123,0.065124,0.065123,0.065124,0.065124,0
2022-04-26 00:15:00+00:00,0.065094,0.065125,0.065094,0.065125,0.065125,1247552
2022-04-26 00:20:00+00:00,0.065071,0.065071,0.065071,0.065071,0.065071,0
...,...,...,...,...,...,...
2022-04-26 14:30:00+00:00,0.063578,0.063578,0.063515,0.063515,0.063515,15872
2022-04-26 14:35:00+00:00,0.063494,0.063494,0.063386,0.063386,0.063386,30336
2022-04-26 14:40:00+00:00,0.063276,0.063276,0.063225,0.063225,0.063225,540160


In [386]:
def process_for_prediction(data):
    i = -1
    row = []
    if len(pd.DataFrame(data).columns) == 7:
      data = data.iloc[: , 1:]        
    data = np.array(data)
    llst = [0, 1, 2, 3, 5]
    grow = []
    srow = []
    for j in llst:
           srow.append([
               data[i-1][j] - data[i-2][j],
               data[i-2][j] - data[i-3][j],
               data[i-3][j] - data[i-4][j],
               data[i-4][j] - data[i-5][j],
               data[i-5][j] - data[i-6][j],
               data[i-6][j] - data[i-7][j],
               data[i-7][j] - data[i-8][j],
               data[i-8][j] - data[i-9][j],
               data[i-9][j] - data[i-10][j],
               data[i-10][j] - data[i-11][j],
               data[i-11][j] - data[i-12][j]
           ])

    for lst in srow:
            mm = np.array(lst)
            mm = np.reshape(mm, (-1, 1))
            grow.append(scaler(mm))

        
    arr = np.array(grow).flatten()
    row.append(arr)


    grow = []
    srow = []
    llst = []
    data = []
    arr = []
    mm = []

    return np.array(row)
def make_prediction(symbol,timeframe):
  if timeframe == "5m":
    data = yf.download(symbol,period="1d",interval=timeframe)
    lst = [str(i) for i in data.index.values]
    ddata = process_for_prediction(data)
    return(model.predict(ddata))    
  if timeframe == "15m":
    data = yf.download(symbol,period="1d",interval=timeframe)
    lst = [str(i) for i in data.index.values]
    ddata = process_for_prediction(data)
    return(model.predict(ddata))  
  if timeframe == "1h":
    data = yf.download(symbol,period="1d",interval=timeframe)
    lst = [str(i) for i in data.index.values]
    ddata = process_for_prediction(data)
    return(model.predict(ddata))  
  if timeframe == "1d":
    data = yf.download(symbol,period="20d",interval=timeframe)
    lst = [str(i) for i in data.index.values]
    ddata = process_for_prediction(data)
    return(model.predict(ddata))  
  if timeframe == "1wk":
    data = yf.download(symbol,period="max",interval=timeframe)
    lst = [str(i) for i in data.index.values]
    ddata = process_for_prediction(data)
    return(model.predict(ddata))  
  if timeframe == "1mo":
    data = yf.download(symbol,period="max",interval=timeframe)
    lst = [str(i) for i in data.index.values]
    ddata = process_for_prediction(data)
    return(model.predict(ddata))  
  if timeframe == "1m":
    data = yf.download(symbol,period="1d",interval=timeframe)
    lst = [str(i) for i in data.index.values]
    ddata = process_for_prediction(data)
    return(model.predict(ddata))  
  if timeframe == "2m":
    data = yf.download(symbol,period="max",interval=timeframe)
    lst = [str(i) for i in data.index.values]
    ddata = process_for_prediction(data)
    return(model.predict(ddata))  
  if timeframe == "30m":
    data = yf.download(symbol,period="1d",interval=timeframe)
    lst = [str(i) for i in data.index.values]
    ddata = process_for_prediction(data)
    return(model.predict(ddata))  
  if timeframe == "5d":
    data = yf.download(symbol,period="5mo",interval=timeframe)
    lst = [str(i) for i in data.index.values]
    ddata = process_for_prediction(data)
    return(model.predict(ddata))  
  if timeframe == "90m":
    data = yf.download(symbol,period="5d",interval=timeframe)
    lst = [str(i) for i in data.index.values]
    ddata = process_for_prediction(data)
    return(model.predict(ddata))  
  if timeframe == "3mo":
    data = yf.download(symbol,period="max",interval=timeframe)
    lst = [str(i) for i in data.index.values]
    ddata = process_for_prediction(data)
    return(model.predict(ddata))  

In [387]:
make_prediction("btc-usd","5m")

[*********************100%***********************]  1 of 1 completed


array([[0.5138444 , 0.12956947]], dtype=float32)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
